In [ ]:
from quantopian.pipeline import Pipeline, CustomFactor
from quantopian.research import run_pipeline
from quantopian.pipeline.factors import AverageDollarVolume, SimpleMovingAverage, Returns, Latest
from quantopian.pipeline.filters import Q500US, StaticAssets
from quantopian.interactive.data.quandl import yahoo_index_vix
from quantopian.pipeline.filters import StaticAssets
from quantopian.pipeline.data.builtin import USEquityPricing

from quantopian.pipeline.classifiers.morningstar import Sector, SuperSector

# PIPELINE を使うとき
from quantopian.pipeline.data.psychsignal import stocktwits_free as st

from zipline.utils.tradingcalendar import trading_day  
from pandas import date_range 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

class CloseOnN(CustomFactor):  
    # Define inputs
    inputs = [USEquityPricing.close]
    window_length = 2 
    
    def compute(self, today, assets, out, close):  
        out[:] = close[0]
        

# キラキラ銘柄とは
+ stocktwit でよくネタにされている銘柄
+ 過去二年総合と，各月総合，MA30日等を見てみようと思う

In [ ]:
#過去二年間総合TOP100＼(^o^)／

def my_pipe():
    pipe = Pipeline()
    pipe.add(st.bull_bear_msg_ratio .latest, 'bull_bear_msg_ratio')
    pipe.add(st.bull_scored_messages .latest, 'bull_scored_messages')
    pipe.add(st.bear_scored_messages .latest, 'bear_scored_messages')
    pipe.add(st.total_scanned_messages .latest, 'total_scanned_messages')
    pipe.add(st.bearish_intensity .latest, 'bearish_intensity')
    pipe.add(st.bullish_intensity .latest, 'bullish_intensity')
    pipe.add(Sector(), 'sector')
    
    pipe.set_screen( Q500US())
    return pipe


def only_spy():
    pipe = Pipeline()
    pipe.add(st.bull_bear_msg_ratio .latest, 'bull_bear_msg_ratio')
    pipe.add(st.bull_scored_messages .latest, 'bull_scored_messages')
    pipe.add(st.bear_scored_messages .latest, 'bear_scored_messages')
    pipe.add(st.total_scanned_messages .latest, 'total_scanned_messages')
    pipe.add(st.bearish_intensity .latest, 'bearish_intensity')
    pipe.add(st.bullish_intensity .latest, 'bullish_intensity')
#     pipe.add(Sector(), 'sector')
    
    spy = StaticAssets(symbols(['SPY']))
    pipe.set_screen(spy)
    return pipe

    

In [ ]:
pipe = my_pipe()
#pipe = only_spy()
pipe_output =  run_pipeline(pipe, start_date="2016-1-1", end_date="2017-8-1")


# ココ一年で一番StockTwitでキラキラさらた銘柄は！

In [ ]:
idx = pd.IndexSlice
l = list()
for sid in pipe_output.index.get_level_values(1).unique():
    l.append((sid,
              pipe_output.loc[idx[:, sid,], "total_scanned_messages"].sum(),
              pipe_output.loc[idx[:, sid,], "bullish_intensity"].mean(),
              pipe_output.loc[idx[:, sid,], "bearish_intensity"].mean(),
              
             ))
df = pd.DataFrame(l, columns=["sid", "total_scanned_messages","bullish_intensity","bearish_intensity"])
df = df.set_index("sid")
df.sort_values(by="total_scanned_messages",ascending=False).head(10)

# AMDさんでした．おめでとうございますー＼(^o^)／

In [ ]:
sids = df.sort_values(by="total_scanned_messages",ascending=False).head(10).index
sids

#Top10のマンスリーの動きを見てみよう

In [ ]:
df_msgs = pipe_output["total_scanned_messages"].unstack(level=-1)
df_msgs.groupby(pd.TimeGrouper(freq='M')).sum()[sids].ix[:"2017-07"].plot(kind='bar')


#AMDさんの話は今年になってからですね．理由はアレですね．四半期で見てもすごいです．

In [ ]:
df_msgs.groupby(pd.TimeGrouper(freq='BQ')).sum()[sids].plot(kind='bar')